**Official implementation of paper "Revisiting In-context Learning Inference Circuit in Large Language Models" (ICLR 2025)**:

### The Kernel Alignment for the Copy Magnitude

This experiment is to calculate the kernel alignment of the hidden states of forerunner tokens and the label token in the next layer to get the Fig. 5 (Left).

Author: Hakaze Cho, yfzhao@jaist.ac.jp, 2024/08

Organized, commented, and modified by: Hakaze Cho, 2025/01/26

**Part I: Import, Define, and Load Everything**

What you should do:
1. [Cell 1] Change to the path from your working directory to the directory containing the README.md file.
2. [Cell 2] Define your experiment parameters.
3. Run the Cell 1 - 4.


In [1]:
# Cell 1: Import libraries and change the working directory.

## Change the working directory
import os
try:
    # Change to the path from your working directory to the directory containing the README.md file.
    os.chdir("ICL_Inference_Dynamics_Released") 
except:
    print("Already in the correct directory or the directory does not exist.")

## Import libraries
from util import load_model_and_data, kernel_alignment, inference
import StaICC
import matplotlib.pyplot as plt

## Some definations for the plots.
plt.style.use('default')
plt.rc('font',family='Cambria Math')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Cambria Math'] + plt.rcParams['font.serif']

In [5]:
# Cell 2: Model and huggingfacetoken configurations

## The huggingface model name to be tested as the LM for ICL. 
## Recommended: "meta-llama/Meta-Llama-3-8B", "tiiuae/falcon-7b", "meta-llama/Meta-Llama-3-70B", "tiiuae/falcon-40b"
ICL_model_name = "tiiuae/falcon-7b" 

## Whether to use the quantized version of the model. 
## Recommended: Keep it default.
quantized = False if ICL_model_name in ["meta-llama/Meta-Llama-3-8B", "EleutherAI/pythia-6.9b", "tiiuae/falcon-7b"] else True

## The huggingface token to access the model. If you use the Llama model, you need to set this.
huggingface_token = "your token here"

# Experiment parameters

## The demonstration numbers. Recommended: 0, 1, 2, 4, 8, 12.
k = 4 

## The used dataset index from the StaICC library. Alternative: 0, 1, 2, 3, 4, 5. See the README.md for more information.
dataset_index = 2 

## Force the ICL_model to reload, even the ICL_model is already in the variables. 
## Recommended: False.
model_forced_reload = False

In [ ]:
# Cell 3: Load the data and build the test inputs.

bench = StaICC.Normal(k)
load_model_and_data.set_abstract_label_space(bench[dataset_index])
prompts, queries = load_model_and_data.load_data_from_StaICC_experimentor(bench[dataset_index], "label_words")

In [ ]:
# Cell 4: Load the model.

vars_dict = vars() if "ICL_model" in vars() else locals()
if "ICL_model" not in vars_dict or model_forced_reload:
    ICL_model, ICL_tknz = load_model_and_data.load_ICL_model(ICL_model_name, huggingface_token = huggingface_token, quantized = quantized)
    loaded = True

**Part II: Run the Experiment**

What you should do:

1. Run the Cell 5 - 6.

In [ ]:
# Cell 5: Get the ICL hidden states from the last 2 tokens (label token, forerunner token) of the prompt.

ICL_hidden_states = inference.ICL_inference_to_multi_token_hidden_states(ICL_model, ICL_tknz, prompts, [-1, -2])

In [ ]:
# Cell 6: Calculate the similarity map and the kernel alignment. Refer to `util/kernel_alignment.py` for more details.

## Calculate the similarity map (defined as $\delta: \mathcal{X}\rightarrow\mathbb{H}^{d}$ in the Appendix A.2) of the label tokens' hidden states, indexed by layer.
label_token_sim_graph = []
for layer_hidden_state in ICL_hidden_states[0]:
    label_token_sim_graph.append(kernel_alignment.sim_graph(layer_hidden_state))

## Calculate the similarity map (defined as $\delta: \mathcal{X}\rightarrow\mathbb{H}^{d}$ in the Appendix A.2) of the forerunner tokens' hidden states, indexed by layer.
forerunner_token_sim_graph = []
for layer_hidden_state in ICL_hidden_states[1]:
    forerunner_token_sim_graph.append(kernel_alignment.sim_graph(layer_hidden_state))

## Calculate the kernel alignment.
### The organization of the results: res_kernel_alignment[layer_index]: (mean, std, individual_values)
res_kernel_alignment = []
for layer in range(len(label_token_sim_graph) - 1):
    res_kernel_alignment.append(kernel_alignment.kernel_alignment(label_token_sim_graph[layer + 1], forerunner_token_sim_graph[layer]))

**Part III: Plot and Save the Result**

What you should do:

1. Run the Cell 7 and 8. You can define your own file name and dictionary to save the result in Cell 8.

In [ ]:
# Cell 7: Data preview.

avg_kernel_alignment_for_plot = []

for line in res_kernel_alignment:
    avg_kernel_alignment_for_plot.append(line[0])

plt.figure(figsize=(4, 3))
plt.xlabel("Transformer Block Number", fontsize = 12)
plt.ylabel("Kernel Alignment", fontsize = 12)
plt.title("Kernel Alignment on dataset " + str(dataset_index) + "\n model: " + ICL_model_name + ", k: " + str(k) , fontsize = 12)
plt.plot(avg_kernel_alignment_for_plot, color="blue")
plt.axhline(0.125, color = "black", linestyle = "--", linewidth = 1, label = "Random Baseline")
plt.legend(loc = 4, prop={'size': 9})

In [11]:
# Cell 8: Save the result.
# Result file organization:
# res_kernel_alignment[layer_number] = (alignment mean, alignment std, alignment of invidivual sample)

import pickle

data_file_name = "data/" + ICL_model_name.replace('/', '_')+ ",cross_alignment_copy," + "," + str(k) + "," + str(dataset_index + 1) + "," + ".pickle"
with open(data_file_name, 'wb') as f:
    pickle.dump(res_kernel_alignment, f)